In [ ]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [11]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_coronal_syntheticMRI2D.ipynb


In [12]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [13]:
from scipy import spatial
from sklearn.decomposition import PCA

In [ ]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [ ]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [16]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [ ]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [27]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,558183.99,14.48,0.44,-4.06,-0.18,-0.10,0.00
4,14.48,776467.92,14.40,2.67,2.32,-0.24,-0.01
5,0.44,14.40,191955.79,0.64,-0.46,0.15,0.01
6,-4.06,2.67,0.64,50677.87,-0.35,-0.05,0.01
7,-0.18,2.32,-0.46,-0.35,3943.91,0.01,0.00
8,-0.10,-0.24,0.15,-0.05,0.01,113.78,-0.00
9,0.00,-0.01,0.01,0.01,0.00,-0.00,2.23


In [33]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00002,0.00000,-0.00002,-0.00000,-0.00001,0.00000
4,0.00002,1.00000,0.00004,0.00001,0.00004,-0.00003,-0.00001
5,0.00000,0.00004,1.00000,0.00001,-0.00002,0.00003,0.00001
6,-0.00002,0.00001,0.00001,1.00000,-0.00003,-0.00002,0.00003
7,-0.00000,0.00004,-0.00002,-0.00003,1.00000,0.00002,0.00000
8,-0.00001,-0.00003,0.00003,-0.00002,0.00002,1.00000,-0.00000
9,0.00000,-0.00001,0.00001,0.00003,0.00000,-0.00000,1.00000


In [34]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

29.899745485669314

In [35]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.86503361e+10 4.24002832e+10 4.89828757e+09 3.19511175e+08
 1.72735460e+06 1.07901110e+03 1.31275786e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.225432,0.973611,-0.034492,-0.008517,-0.000657,-0.000019,-3.911163e-07
1,0.970988,0.221540,-0.087741,-0.020020,-0.001541,-0.000045,-8.754454e-07
2,0.076310,0.052207,0.993031,-0.072889,-0.005182,-0.000149,-2.913111e-06
3,0.023095,0.016542,0.070379,0.996761,-0.026573,-0.000761,-1.472469e-05
4,0.002357,0.001691,0.006857,0.026069,0.999528,-0.014456,-2.828281e-04
5,0.000102,0.000073,0.000295,0.001123,0.014421,0.999703,-1.959421e-02
6,0.000004,0.000003,0.000012,0.000044,0.000565,0.019588,9.998080e-01


In [36]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.2254320288782992,
 0.7784596930274258,
 0.006968517872579794,
 0.0032387224195972175,
 0.00047213906843945885,
 0.0002966798168190232,
 0.00019202505595750097]